In [16]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
# Importando o módulo de Regressão Linear do scikit-learn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from Análise_de_Correlação import analiseCorr
from Normalização import normalizaDf
df= pd.read_excel("input_acoes_NotasFundamentei_20200517.xlsx")
abas= pd.ExcelFile("input_planilhaCanalHolder_20200211.xlsx")
MA=abas.parse('Múltiplos Ações')
FA=abas.parse('Fundamentos Ações')

In [17]:
#Join das tabelas "Fundamentos ações" e "NotasCanalFundamentei" e um merge entre as duas primeiras abas do arquivo input_planilhaCanalHolder_20200211.xlsx
MA=MA.join(df.set_index("Código de Neg."),on="Código de Neg.")
uniaoAbas = pd.merge(MA, FA, on="Nome", how='outer', suffixes=('', '_y'))
uniaoAbas = uniaoAbas.drop(list(uniaoAbas.filter(regex='_y$')), axis=1)
#Remoção de uma coluna inútil
uniaoAbas.drop("Unnamed: 0",axis=1,inplace=True)

In [18]:
#Removando uma coluna de notas que nao foi utilizada, pois tem dados faltando
uniaoAbas.drop("NotaFundamentei20200201",axis=1,inplace=True)

In [19]:
#Selecionando todas as empresas que possuem notas e estão com seus dados completos
empresasComNotas=uniaoAbas.replace("-",np.nan)
empresasComNotas.dropna(inplace=True)

In [129]:
avaliação=[]
for k in empresasComNotas["NotaFundamentei"]:
    if k>=3.0:
        avaliação.append("Boa")
    #elif (k>=3) & (k<4):
     #   avaliação.append("Mediana")
    else:
        avaliação.append("Ruim")

In [130]:
empresasComNotas['Avaliação'] = avaliação
empresasComNotas[["NotaFundamentei","Avaliação"]]

,NotaFundamentei,Avaliação
31,4.6,Boa
39,4.5,Boa
63,2.6,Ruim
84,2.4,Ruim
101,3.4,Boa
102,3.2,Boa
111,3.7,Boa
117,3.3,Boa
119,2.4,Ruim
133,4.4,Boa


In [131]:
dfCNota=normalizaDf(empresasComNotas)

In [132]:
#Gerando a analise de correlação dos atributos, já excluindo a "NotaFundamentei" da resposta
analise=analiseCorr("NotaFundamentei",dfCNota)[1:]

In [133]:
#Selecionando os melhores atributos para treinar o regressor
melhoresAtributos=[]
count=0
for k in analise.values:
    if (k > 0.3) | (k< -0.3):
        melhoresAtributos.append(analise.axes[0][count])
    count+=1

In [134]:
X,y=dfCNota[melhoresAtributos], empresasComNotas["Avaliação"]
melhoresAtributos

['EV/EBITDA',
 'P/VPA',
 'Cotação',
 2018,
 'Caixa e Equivalentes',
 'Receita Operacional',
 'Lucro Bruto',
 'Caixa Líquido',
 2016,
 'Dívida Bruta',
 'Patrimônio Líquido',
 'Patrimônio',
 'Passivo Circulante',
 2017,
 'Ativo Circulante',
 'FCO',
 'Beta',
 'Ativo Não Circulante',
 'Fluxo de Caixa Operacional (FCO)',
 'Passivo Não Circulante']

In [135]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=0)
y_test

149     Boa
277     Boa
136     Boa
63     Ruim
228     Boa
178     Boa
356     Boa
322    Ruim
263    Ruim
175     Boa
Name: Avaliação, dtype: object

In [136]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [137]:
metricas = ["accuracy","balanced_accuracy","average_precision"]

In [138]:
model.get_params().keys()

dict_keys(['objective', 'base_score', 'booster', 'colsample_bylevel', 'colsample_bynode', 'colsample_bytree', 'gamma', 'gpu_id', 'importance_type', 'interaction_constraints', 'learning_rate', 'max_delta_step', 'max_depth', 'min_child_weight', 'missing', 'monotone_constraints', 'n_estimators', 'n_jobs', 'num_parallel_tree', 'random_state', 'reg_alpha', 'reg_lambda', 'scale_pos_weight', 'subsample', 'tree_method', 'validate_parameters', 'verbosity'])

In [149]:
parameters = {'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

'\neta[0.2,0,01];\n\n'

In [159]:
from sklearn.model_selection import GridSearchCV
clf = GridSearchCV(model, parameters, cv=5)

In [162]:
clf.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, gamma=None,
                                     gpu_id=None, importance_type='gain',
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs...
                                     num_parallel_tree=None, random_state=None,
                                     reg_alpha=None, reg_lambda=None,
                                     scale_pos_weight=None, subsample=None,
                                     tree_method=None, va

In [163]:
clf.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=11, missing=-999, monotone_constraints='()',
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              random_state=1337, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=1337, subsample=0.8, tree_method='exact',
              validate_parameters=1, verbosity=None)

In [164]:
Y_prev=clf.predict(X_test)

In [165]:
Y_prev

array(['Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa', 'Boa',
       'Boa'], dtype=object)

In [166]:
from sklearn.metrics import accuracy_score,classification_report

# Avaliar a precisão
print("Acurácia: ",accuracy_score(y_test, Y_prev))

Acurácia:  0.7


In [167]:
print(classification_report(y_test, Y_prev))

              precision    recall  f1-score   support

         Boa       0.70      1.00      0.82         7
        Ruim       0.00      0.00      0.00         3

    accuracy                           0.70        10
   macro avg       0.35      0.50      0.41        10
weighted avg       0.49      0.70      0.58        10



c:\users\danie\appdata\local\programs\python\python38\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [168]:
print("Acurácia balanceada: ",(1)/2)

Acurácia balanceada:  0.5


In [169]:
#Criando um outro dataFrame para vizualizar NotaFundamentei x NotaPredita 
emp=empresasComNotas.loc[X_test.index.values,["Nome","Código de Neg.","NotaFundamentei","Avaliação"]]
emp["AvaliaçãoPredita"]=Y_prev
emp.sort_values("NotaFundamentei",ascending=False)

,Nome,Código de Neg.,NotaFundamentei,Avaliação,AvaliaçãoPredita
136,Engie Brasil,EGIE3,4.8,Boa,Boa
228,Lojas Renner,LREN3,4.8,Boa,Boa
149,Fleury,FLRY3,4.6,Boa,Boa
175,Hypera,HYPE3,4.3,Boa,Boa
277,Qualicorp,QUAL3,3.6,Boa,Boa
178,Iguatemi,IGTA3,3.3,Boa,Boa
356,Totvs,TOTS3,3.3,Boa,Boa
322,Smiles,SMLS3,2.7,Ruim,Boa
263,Petrobras,PETR3,2.7,Ruim,Boa
63,BR Malls Par,BRML3,2.6,Ruim,Boa
